In [1]:
import pandas as pd

uid_check = ['아파트1-104-1006']


In [2]:
total_ecv = pd.DataFrame(columns=['UECV', 'SORTECV', "UCPDV", "SORTCPDV"])

In [3]:
# uid_check.remove('아파트3-103-312')
len(uid_check)

1

In [4]:
import pandas as pd
from modules.UCKMeans import UCKMeans as UC
from modules.UCOSKMeans import UCOSKMeans as UCOS
from modules.SortMeans import SortMeans as SM

# Mongo 연결
from pymongo import MongoClient as mc
import random
import pprint as pp

#  print(uid_check)
# custom module
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

mongo_uri = "mongodb://localhost:27017"
client = mc(mongo_uri)

keti_pr_db = client.keti_pattern_recognition
household_col = keti_pr_db.household_info

for seq in range(0, 64):
    total = household_col.find({"uid": { "$nin" : uid_check }}).count()
    while True:
        db_datas = list(household_col.\
                        find({
                            "uid": { "$nin" : uid_check }
                            }).skip(random.randrange(0,total)).limit(1))
        if len(db_datas) == 0 :
            continue
        else:
            db_datas = db_datas[0]
            break;
    # db_datas = household_col.\
    #                 find_one({
    #                     "uid": "아파트4-103-606"
    #                 })


    # require module
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    from datetime import datetime as dt

    sns.set(rc={'figure.figsize': (15.7, 13.27)})
    plt.rcParams['figure.figsize'] = 15.7,13.27
    plt.rcParams['font.family'] = 'AppleGothic'

    # custom module
    %load_ext autoreload
    %autoreload 2
    %reload_ext autoreload

    import custom.supports as sp
    import custom.mapgenerator as mg
    import custom.dateutils as du

    SEASONS = {
        "봄" : [3, 4, 5],
        "여름" : [6, 7, 8],
        "가을" : [9, 10, 11],
        "겨울" : [12, 1, 2]
    }

    SEASONSARR = ['봄','여름','가을','겨울']

    '''
     0 : 봄
     1 : 여름
     2 : 가을
     3 : 겨울
    '''

    DAYARR = ["월", "화", "수", "목", "금", "토", "일"]

    uid, timeslot = db_datas['uid'], db_datas['timeslot']
    uid_check.append(uid)

    print("{}:{} pattern recognition".format(seq,uid))

    datelist = [
        dt.strptime(ts['time'], "%Y-%m-%d T%H:%M %z").date()
        for ts in timeslot
    ]
    datelist = list(set(datelist))
    datelist.sort()

    ts_datas = {}
    start_idx = 0
    end_idx = 96
    enl = 1000

    for date in datelist:
        ts_datas[date] = [ ts['power'] * enl for ts in timeslot[start_idx:end_idx]]
        start_idx = end_idx
        end_idx = end_idx + 96

    ts_datas = pd.DataFrame(ts_datas).T
    datas = ts_datas.reset_index().copy()

    datas.rename(columns={"index": "date"}, inplace=True)
    datas['date'] = pd.to_datetime(datas['date'])
    datas['month'] = [dt.month for dt in datas['date']]
    datas = [
        datas[(datas['month'].isin(SEASONS[season]))].copy()
        for season in SEASONSARR
    ]

    bak_datas = datas.copy()

    season_datas = {
        "봄": datas[0].copy(),
        "여름": datas[1].copy(),
        "가을": datas[2].copy(),
        "겨울": datas[3].copy()
    }

    # merge timeslot
    season = "가을"

    timeslot = 1
    merge_size = timeslot * 4
    merge_datas = season_datas[season].copy()
    merge_datas = merge_datas[
        merge_datas.columns.difference(['month', 'date'])
    ]
    merge_datas.index = season_datas[season]['date']

    tmp = pd.DataFrame()
    start_idx = 0
    end_idx = 96
    for date in merge_datas.index:
        in_arr = []
        og_arr = merge_datas.loc[date].values
        for merging in range(0, int(end_idx / merge_size)):
            in_arr.append(og_arr[
                (merging) * merge_size
                :
                (merging + 1) * merge_size
            ].sum())
    #     logging
    #     print("date end : {}, size: {}".format(date, len(in_arr)))

        tmp[date] = in_arr
    merge_datas = tmp.copy()

    merge_datas
    
    
    uc = UC(merge_datas, K=10)
    uc.run()
    
#     ucos = UCOS(merge_datas, K=10)
#     ucos.run()

    sm = SM(merge_datas, K=10)
    sm.run()
    
    total_ecv.loc[uid] = [
        uc.get_ECV(),
        sm.get_ECV(),
        uc.get_CPDV(),
        sm.get_CPDV()
    ]
    
    print("now : {}".format(total_ecv.mean(axis=0)))
    
client.close()
total_ecv

<ipython-input-4-a04476921270>:24: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total = household_col.find({"uid": { "$nin" : uid_check }}).count()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
0:아파트1-104-902 pattern recognition
---UCKMeans Init---
K:10
---Go UCMeans---
[]
-------Remove One Pattern Success-------
  Target length 91 -> 91
DIS_INFO {'Q1': 0.18512145588854106, 'Q3': 0.25592593979486655, 'IQR': 0.07080448390632549, 'step': 0.10620672585948823}
DatetimeIndex(['2018-10-15', '2018-10-18', '2018-10-21', '2018-10-26',
               '2018-10-28', '2018-10-29', '2018-11-01', '2018-11-03',
               '2018-11-05', '2018-11-09', '2018-11-15', '2018-11-16',
               '2018-11-21', '2018-11-24', '2018-11-27', '2018-11-29',
               '2018-11-30'],
              dtype='datetime64[ns]', freq=None)
-------Remove Outlier Success-------
  Target length 91 -> 74
---Now 0---
TSS: 9538828.216216216, WSS: 7546240.0, ECV: 20.88923472621902, CPDV: 0.9312881810043029
---Now 1---
TSS: 9538828.216216216, WSS: 2807582.513798539, ECV: 70.56679866584045, CPDV: 0.9590704916448789
---Now 2--

<ipython-input-4-a04476921270>:24: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total = household_col.find({"uid": { "$nin" : uid_check }}).count()


---UCKMeans Init---
K:10
---Go UCMeans---
[]
-------Remove One Pattern Success-------
  Target length 91 -> 91
DIS_INFO {'Q1': 0.11214608785052571, 'Q3': 0.2797137857362786, 'IQR': 0.16756769788575288, 'step': 0.2513515468286293}
DatetimeIndex(['2018-09-14', '2018-09-26', '2018-10-03', '2018-10-18'], dtype='datetime64[ns]', freq=None)
-------Remove Outlier Success-------
  Target length 91 -> 87
---Now 0---
TSS: 15312679.839080459, WSS: 16422971.0, ECV: -7.250795893256368, CPDV: 0.9145161569898442
---Now 1---
TSS: 15312679.839080459, WSS: 8224066.983426558, ECV: 46.29243822862804, CPDV: 0.9589223232422758
---Now 2---
TSS: 15312679.839080459, WSS: 7444774.085547241, ECV: 51.381638199298315, CPDV: 0.9630230874170371
---Now 3---
TSS: 15312679.839080459, WSS: 7060653.54029112, ECV: 53.89015107420205, CPDV: 0.9646342623414539
---Now 4---
TSS: 15312679.839080459, WSS: 6968504.201187551, ECV: 54.49193560879664, CPDV: 0.9650853022492792
---Now 5---
TSS: 15312679.839080459, WSS: 6968504.2011875

<ipython-input-4-a04476921270>:24: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total = household_col.find({"uid": { "$nin" : uid_check }}).count()


...

<ipython-input-4-a04476921270>:24: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total = household_col.find({"uid": { "$nin" : uid_check }}).count()


---UCKMeans Init---
K:10
---Go UCMeans---
[]
-------Remove One Pattern Success-------
  Target length 91 -> 91
DIS_INFO {'Q1': 0.07613892895394639, 'Q3': 0.21713199206250755, 'IQR': 0.14099306310856116, 'step': 0.21148959466284173}
DatetimeIndex(['2018-09-23', '2018-09-24', '2018-10-13', '2018-10-14',
               '2018-11-10', '2018-11-11', '2018-11-25'],
              dtype='datetime64[ns]', freq=None)
-------Remove Outlier Success-------
  Target length 91 -> 84
---Now 0---
TSS: 22569473.94047619, WSS: 21332613.0, ECV: 5.48023823567283, CPDV: 0.9593870409754977
---Now 1---
TSS: 22569473.94047619, WSS: 11332234.667119596, ECV: 49.789548941163766, CPDV: 0.9776074904002359
---Now 2---
TSS: 22569473.94047619, WSS: 11025789.898861514, ECV: 51.14733321680212, CPDV: 0.9783846536986135
---Now 3---
TSS: 22569473.94047619, WSS: 10808028.138448773, ECV: 52.1121840635124, CPDV: 0.9788273524762956
---Now 4---
TSS: 22569473.94047619, WSS: 10741138.283290884, ECV: 52.40855718826622, CPDV: 0.9788

<ipython-input-4-a04476921270>:24: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  total = household_col.find({"uid": { "$nin" : uid_check }}).count()


---UCKMeans Init---
K:10
---Go UCMeans---
[]
-------Remove One Pattern Success-------
  Target length 91 -> 91
DIS_INFO {'Q1': 0.13287737992727433, 'Q3': 0.3326957457539249, 'IQR': 0.19981836582665058, 'step': 0.29972754873997587}
DatetimeIndex(['2018-09-11', '2018-09-22', '2018-09-23', '2018-09-27',
               '2018-10-02', '2018-10-21', '2018-10-30', '2018-11-14'],
              dtype='datetime64[ns]', freq=None)
-------Remove Outlier Success-------
  Target length 91 -> 83
---Now 0---
TSS: 79207678.89156622, WSS: 94298875.0, ECV: -19.052693273708154, CPDV: 0.9781535435010817
---Now 1---
TSS: 79207678.89156622, WSS: 41427755.10721151, ECV: 47.697299445013016, CPDV: 0.9865341242001363
---Now 2---
TSS: 79207678.89156622, WSS: 38563082.66008068, ECV: 51.313959454773574, CPDV: 0.9869616032229485
---Now 3---
TSS: 79207678.89156622, WSS: 37454704.878869906, ECV: 52.71329067710131, CPDV: 0.9871769540916354
---Now 4---
TSS: 79207678.89156622, WSS: 37118622.682348624, ECV: 53.137595746034

,UECV,SORTECV,UCPDV,SORTCPDV
아파트1-104-902,73.646305,73.451039,0.963311,0.962995
아파트2-5-1502,54.491936,53.960555,0.965085,0.965323
아파트3-102-702,44.142455,46.089112,0.944187,0.956930
아파트2-5-1001,53.460845,41.196208,0.948248,0.943553
아파트1-103-1002,49.958447,45.775747,0.992128,0.991753
...,...,...,...,...
아파트1-102-501,38.681087,39.573091,0.969966,0.971202
아파트1-103-1502,52.063828,52.008587,0.986058,0.986313
아파트1-104-606,39.328429,40.649111,0.968367,0.971646
아파트3-102-201,52.788912,57.095718,0.979026,0.980857


In [51]:
total_ecv["ecv_win"] = \
    ["UECV" if total_ecv.loc[idx]['UECV'] > total_ecv.loc[idx]['SORTECV'] else 
        "DRAW" if total_ecv.loc[idx]['UECV'] == total_ecv.loc[idx]['SORTECV'] else
         "SORTECV" for idx in total_ecv.index]
total_ecv["cpdv_win"] = \
    ["UCPDV" if total_ecv.loc[idx]['UCPDV'] > total_ecv.loc[idx]['SORTCPDV'] else 
     "DRAW" if total_ecv.loc[idx]['UCPDV'] == total_ecv.loc[idx]['SORTCPDV'] else
     "SORTCPDV" for idx in total_ecv.index]

total_ecv["ecv_diff"] = \
    [(total_ecv.loc[idx]['UECV'] - total_ecv.loc[idx]['SORTECV'])
         if total_ecv.loc[idx]['UECV'] >= total_ecv.loc[idx]['SORTECV'] else
             (total_ecv.loc[idx]['SORTECV'] - total_ecv.loc[idx]['UECV']) for idx in total_ecv.index]
total_ecv["cpdv_diff"] = \
    [(total_ecv.loc[idx]['UCPDV'] - total_ecv.loc[idx]['SORTCPDV'])
         if total_ecv.loc[idx]['UCPDV'] >= total_ecv.loc[idx]['SORTCPDV'] else
             (total_ecv.loc[idx]['SORTCPDV'] - total_ecv.loc[idx]['UCPDV']) for idx in total_ecv.index]

total_ecv

,UECV,SORTECV,UCPDV,SORTCPDV,ecv_win,cpdv_win,ecv_diff,cpdv_diff
아파트2-2-1510,53.407927,51.727227,0.959633,0.961722,UECV,SORTCPDV,1.680700,0.002089
아파트2-1-1204,42.995614,45.292711,0.969764,0.971281,SORTECV,SORTCPDV,2.297098,0.001517
아파트1-104-507,54.747483,54.002987,0.983947,0.985711,UECV,SORTCPDV,0.744495,0.001764
아파트3-103-1012,40.604517,40.604517,0.953021,0.953021,DRAW,DRAW,0.000000,0.000000
아파트1-101-1602,46.025553,46.947379,0.961189,0.962816,SORTECV,SORTCPDV,0.921826,0.001627
...,...,...,...,...,...,...,...,...
아파트3-103-213,54.952245,54.732287,0.972746,0.973605,UECV,SORTCPDV,0.219958,0.000859
아파트4-103-1206,40.587336,40.587336,0.964129,0.964129,DRAW,DRAW,0.000000,0.000000
아파트2-4-1206,46.762363,47.746177,0.976772,0.978930,SORTECV,SORTCPDV,0.983814,0.002158
아파트3-101-1405,39.934697,40.898960,0.965709,0.967437,SORTECV,SORTCPDV,0.964262,0.001728


In [52]:
print("""---Uclidean Based Diff Max---
    ECV Win Count : {}
    ECV Diff Max : {}
    CPDV Win Count : {}
    CPDV Diff Max : {}
""".format(
total_ecv.groupby(total_ecv['ecv_win']).count().loc['UECV'].values[0],
total_ecv[
    total_ecv['ecv_win'] == "UECV"
]['ecv_diff'].max(),
total_ecv.groupby(total_ecv['cpdv_win']).count().loc['UCPDV'].values[0],
total_ecv[
    total_ecv['cpdv_win'] == "UCPDV"
]['cpdv_diff'].max()))

print("""---Uclidean + Cosine Based (Sort Based) Diff Max---
    ECV Win Count : {}
    ECV Diff Max : {}
    CPDV Win Count : {}
    CPDV Diff Max : {}
""".format(
total_ecv.groupby(total_ecv['ecv_win']).count().loc['SORTECV'].values[0],
total_ecv[
    total_ecv['ecv_win'] == "SORTECV"
]['ecv_diff'].max(),
total_ecv.groupby(total_ecv['cpdv_win']).count().loc['SORTCPDV'].values[0],
total_ecv[
    total_ecv['cpdv_win'] == "SORTCPDV"
]['cpdv_diff'].max()))

print("""---DRAW COUNT---
    ECV : {}
    CPDV : {}
""".format(
total_ecv.groupby(total_ecv['ecv_win']).count().loc['DRAW'].values[0],
total_ecv.groupby(total_ecv['cpdv_win']).count().loc['DRAW'].values[0]))

---Uclidean Based Diff Max---
    ECV Win Count : 22
    ECV Diff Max : 6.882169777612113
    CPDV Win Count : 5
    CPDV Diff Max : 0.0010815747490452443

---Uclidean + Cosine Based (Sort Based) Diff Max---
    ECV Win Count : 32
    ECV Diff Max : 7.25464853970594
    CPDV Win Count : 49
    CPDV Diff Max : 0.009294752561708752

---DRAW COUNT---
    ECV : 10
    CPDV : 10



In [42]:
# 이상치들 기록
bak_total = total_ecv[
    total_ecv['ecv_diff'] >= 20
]

bak_total

,UECV,SORTECV,UCPDV,SORTCPDV,ecv_win,cpdv_win,ecv_diff,cpdv_diff
아파트4-103-909,97.441906,76.745362,0.796317,0.779577,UECV,UCPDV,20.696544,0.01674


In [56]:
from statistics import median